# Generation of Random Clusters with Specified Degree of Separation

In [1]:
import numpy as np
from numpy import linalg as LA
from scipy import optimize as opt
from scipy.stats import ortho_group
import matplotlib as plt
import random

from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.cluster import DBSCAN, KMeans
from sklearn.mixture import GaussianMixture
from kemlglearn.cluster.consensus import SimpleConsensusClustering
from pylab import *
import seaborn as sns
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib notebook

## The main (and only) function to call to generate the dataset of clusters

In [2]:
def make_sepclusters(n_features, n_clusters, n_noisy=0, sep_degree=0.2, sep_quantile=1.96, sep_error = 0.001, outliers=0,
                     lambda_min=1, lambda_ratio=10, min_cluster_size=10, max_cluster_size=100):
    y=[]
    
    # Step 2: Generate cluster centers and random cvariance matrices in non-noisy dimensions 
    # satisfying separation parameter
    centers, Covariances = cluster_center_allocation(n_features, n_clusters, sep_degree, sep_quantile, 
                                                     lambda_min, lambda_ratio, sep_error)
    
    # Step 3: Generate sizes of each cluster and generate memberships of each datapoint
    N_clus = [np.random.randint(min_cluster_size,max_cluster_size + 1) for i in range(n_clusters)]
    N = sum(N_clus)
    for i in range(n_clusters):
        for j in range(N_clus[i]):
            y.append(i)
    
    # Step 4: Generate the mean vector and covariance matrix of the noisy variables
    indices = []
    if n_noisy > 0:
        p = np.array(N_clus) / N
        Cov_mix = np.zeros(Covariances[0].shape)
        cent_mix = np.zeros(centers[0].shape)
        for i in range(len(Covariances)):
            Cov_mix += p[i]*Covariances[i]
            cent_mix += p[i]*centers[i]

        for kp in range(len(Covariances)):
            for k in range(kp):
                Cov_mix += p[k]*p[kp] * np.matmul((centers[k] - centers[kp]),np.transpose(centers[k] - centers[kp]))

        min_cent_mix = min(cent_mix)
        max_cent_mix = max(cent_mix)
        center_noisy = np.array([np.random.uniform(min_cent_mix,max_cent_mix) for i in range(n_noisy)])
        eigenvalues_mix, _ = LA.eig(Cov_mix)
        eigenvalues_mix = np.sort(eigenvalues_mix)

        Covariance_noisy = generate_covariance(n_noisy, eigenvalues_mix[0], 
                                               eigenvalues_mix[n_noisy-1]/eigenvalues_mix[0])
    perm = np.random.permutation(n_features + n_noisy)
            
        
    # Step 5: Apply a random rotation to the cluster means and covariance matrices in Step 2
    rotation = ortho_group.rvs(n_features)
    
    # Step 7: Generate random vectors for each of the K clusters from a given family of elliptical distributions.
    if outliers < 1:
        X = np.zeros([N+int(outliers*N),n_features+n_noisy])
    else:
        X = np.zeros([N+outliers,n_features+n_noisy])
    ind = 0
    for i in range(n_clusters):
        for j in range(N_clus[i]):
            x = np.random.multivariate_normal(centers[i][:,0], Covariances[i])
            x = np.transpose(np.matmul(rotation, np.transpose(x)))
            if n_noisy > 0:
                x = np.append(x, np.random.multivariate_normal(center_noisy[:,0], Covariance_noisy))
            X[ind,:] = x
            ind += 1
    
    X = X[:,perm]
    
    # Step 8: Calculate the population separation index matrices and projection directions for pairs of clusters 
    # via the population mean vectors and covariance matrices.
    
    J = np.ones([n_clusters,n_clusters])
    A = [[np.zeros([n_features+n_noisy,1]) for j in range(n_clusters)] for i in range(n_clusters)]
    pop_centers = []
    pop_Covariances = []
    
    next_ind = 0
    for i in range(n_clusters):
        pop_centers.append(np.transpose(np.mean(X[next_ind:(next_ind+N_clus[i]),:],0)))
        pop_Covariances.append(np.cov(np.transpose(X[next_ind:(next_ind+N_clus[i]),:])))
        next_ind += N_clus[i]
    
    for i in range(n_clusters):
        for j in range(i+1,n_clusters):
            # Find optimal projection direction
            mu1 = pop_centers[i]
            mu2 = pop_centers[j]
            Sigma1 = pop_Covariances[i]
            Sigma2 = pop_Covariances[j]
            a = optimal_projection(mu1, mu2, Sigma1, Sigma2, sep_quantile)
            A[i][j] = a
            A[j][i] = a
            J[i,j] = calculate_J(mu1, mu2, Sigma1, Sigma2, sep_quantile, a)
            J[j,i] = J[i,j]
    
    #print(J)
    
    
    # Step 10: Generate outliers. The memberships of outliers are assigned as zero
    pop_mean = np.mean(X,0)
    pop_std = np.std(X,0)
    out_min = pop_mean - 4*pop_std
    out_max = pop_mean + 4*pop_std
    for i in range(N, X.shape[0]):
        X[i,:] = np.random.uniform(out_min, out_max)
        y.append(np.nan)
        
    perm_ex = np.random.permutation(X.shape[0])
    
    X = X[perm_ex,:]
    y = np.array(y)
    y = y[perm_ex]
    
    return X, y

## Auxiliary functions

In [3]:
def cluster_center_allocation(n_features, n_clusters, sep_degree, sep_quantile, lambda_min, lambda_ratio, sep_error):
    
    # Step (a): generate K covariance mantrices in n_features dimensions
    Covariances = [generate_covariance(n_features, lambda_min, lambda_ratio) for i in range(n_clusters)]
    
    # Step (b): Construct simplex
    edge = 2
    V = calculate_vertices(edge, n_features)
    
    # Step (c): allocate cluster centers
    centers = calculate_centers(V, n_clusters)
    
    # Step (d): Calculate separation matrix
    J = np.ones([n_clusters,n_clusters])
    A = [[np.zeros([n_features,1]) for j in range(n_clusters)] for i in range(n_clusters)]
    for i in range(n_clusters):
        for j in range(i+1,n_clusters):
            # Find optimal projection direction
            mu1 = centers[i]
            mu2 = centers[j]
            Sigma1 = Covariances[i]
            Sigma2 = Covariances[j]
            a = optimal_projection(mu1, mu2, Sigma1, Sigma2, sep_quantile)
            A[i][j] = a
            A[j][i] = a
            J[i,j] = calculate_J(mu1, mu2, Sigma1, Sigma2, sep_quantile, a)
            J[j,i] = J[i,j]
    
    # Step (e): Scale the length of simplex edge
    min_sep = np.min(J)
    min_edge = 0
    max_edge = np.inf
    while np.abs(min_sep - sep_degree) > sep_error:
        if min_sep < sep_degree:
            min_edge = edge
            if np.isinf(max_edge):
                edge *= 2
            else:
                edge = (edge + max_edge) / 2
        elif min_sep > sep_degree:
            max_edge = edge
            edge = (edge + min_edge) / 2
        
        V = calculate_vertices(edge, n_features)
        centers = calculate_centers(V, n_clusters)
        J = calculate_Js(n_clusters, centers, Covariances, sep_quantile, A)
        min_sep = np.min(J)
    
    # Step (f): Compute separation indeces between custers and their nearest neighbors. If conditions are satisfied - stop,
    # otherwise - Step (g)
    Jk = np.min(J,1)
    k_opt = np.argmax(Jk)
    
    while np.abs(Jk[k_opt] - sep_degree) > sep_error:
        # Step (g): Scale the covariance matrix
        mu1 = centers[k_opt]
        k2_min = np.argmin(J[k_opt,:])
        mu2 = centers[k2_min]
        Sigma1 = Covariances[k_opt]
        Sigma2 = Covariances[k2_min]
        a = A[k_opt][k2_min]
        
        func = (lambda x: (np.matmul(np.transpose(a),(mu2-mu1))-sep_quantile*
                         (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1*x),a)) + 
                          np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))) /
                     (np.matmul(np.transpose(a),(mu2-mu1))+sep_quantile*
                         (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1*x),a)) + 
                          np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))) - sep_degree)
        
        func_obj = (lambda x: np.abs((np.matmul(np.transpose(a),(mu2-mu1))-sep_quantile*
                         (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1*x),a)) + 
                          np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))) /
                     (np.matmul(np.transpose(a),(mu2-mu1))+sep_quantile*
                         (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1*x),a)) + 
                          np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))) - sep_degree))
        
        
        if func(1) > 0:
            res = opt.minimize(func_obj, 1.01, constraints=[{'type':'ineq','fun':(lambda x: 
                               np.matmul(np.matmul(np.transpose(a),Sigma1*x),a))}],
                         bounds=[(1.001,None)])
        else:
            res = opt.minimize(func_obj, 0.99, constraints=[{'type':'ineq','fun':(lambda x: 
                               np.matmul(np.matmul(np.transpose(a),Sigma1*x),a))}],
                         bounds=[(0,0.999)])
        Covariances[k_opt] = Covariances[k_opt]*res.x[0]
        
        J = calculate_Js(n_clusters, centers, Covariances, sep_quantile, A)
        Jk = np.min(J,1)
        k_opt = np.argmax(Jk)
    
    for i in range(n_clusters):
        J[i,i] = -1.
        
    #print(J)
    
    return centers, Covariances

In [4]:
def generate_covariance(n_features, lambda_min, lambda_ratio):
    lambda_min = np.abs(lambda_min)
    lambda_ratio = np.abs(lambda_ratio)
        
    eigenvalues = np.array([np.random.uniform(lambda_min, lambda_min * lambda_ratio) for i in range(n_features)])
    eigenvalues = np.sort(eigenvalues)[::-1]
    L = np.diag(eigenvalues)
    Q = ortho_group.rvs(n_features)
    C = np.matmul(np.matmul(Q, L), np.transpose(Q))
    
    return C

In [5]:
def v_mean(V):
    v = np.array(V[0])
    k = len(V)
    for i in range(1,k):
        v += V[i]
    v /= k
    return v

In [6]:
def next_vertex(V,edge):
    k = len(V)
    p = len(V[0])
    vk = v_mean(V)
    v = np.array(vk)
    for i in range(k,p):
        v[i] = 0
    t = 0
    for i in range(0,k):
        t += np.matmul(np.transpose(V[i] - vk), (V[i] - vk))
    v[k-1] = np.sqrt(edge*edge - t/k)
    return v

In [7]:
def optimal_projection(mu1, mu2, Sigma1, Sigma2, quantile):
    res = opt.minimize(lambda x: -(np.matmul(np.transpose(x),(mu2-mu1))-quantile*
                                      (np.sqrt(np.matmul(np.matmul(np.transpose(x),Sigma1),x)) + 
                                       np.sqrt(np.matmul(np.matmul(np.transpose(x),Sigma2),x)))) /
                                  (np.matmul(np.transpose(x),(mu2-mu1))+quantile*
                                      (np.sqrt(np.matmul(np.matmul(np.transpose(x),Sigma1),x)) + 
                                       np.sqrt(np.matmul(np.matmul(np.transpose(x),Sigma2),x)))),
                 (mu2 - mu1), constraints={'type':'ineq','fun':(lambda x: np.matmul(np.transpose(x),(mu2-mu1)))})
    
    return res.x

In [8]:
def calculate_Js(n_clusters, centers, Covariances, quantile, A):
    J = np.ones([n_clusters,n_clusters])
    for i in range(n_clusters):
        for j in range(i+1,n_clusters):
            mu1 = centers[i]
            mu2 = centers[j]
            Sigma1 = Covariances[i]
            Sigma2 = Covariances[j]
            a = A[i][j]
            J[i,j] = calculate_J(mu1, mu2, Sigma1, Sigma2, quantile, a)
            J[j,i] = J[i,j]
            
    return J

In [9]:
def calculate_J(mu1, mu2, Sigma1, Sigma2, quantile, a):
    J = ((np.matmul(np.transpose(a),(mu2-mu1))-quantile * (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1),a)) + 
           np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))) /
         (np.matmul(np.transpose(a),(mu2-mu1))+quantile * (np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma1),a)) + 
           np.sqrt(np.matmul(np.matmul(np.transpose(a),Sigma2),a)))))
            
    return J

In [10]:
def calculate_vertices(edge,n_features):
    V = []
    V.append(-np.eye(n_features,1)*edge/2)
    V.append( np.eye(n_features,1)*edge/2)
    for i in range(2,n_features + 1):
        V.append(next_vertex(V,edge))
    
    return V

In [11]:
def calculate_centers(V, n_clusters):
    p = len(V[0])
    if n_clusters <= p + 1:
        centers = V[:n_clusters]
    else:
        centers = V
        p1 = p+1
        for i in range(p1, n_clusters):
            centers.append(V[1 + i%p1] + np.eye(p,1) * 2 * int(i/p1))
    
    return centers

# Test datasets generation

In [12]:
X = []
y = []

for s in [0.01,0.21,0.342]:
    for noise in [2,0]:
        XX,yy = make_sepclusters(n_features=4, n_clusters=3, sep_degree=s, n_noisy=noise, min_cluster_size=30, 
                                 max_cluster_size=100, outliers = 20)
        X.append(XX)
        y.append(yy)
    
for s in [0.01,0.21,0.342]:
    for noise in [2,0]:
        XX,yy = make_sepclusters(n_features=4, n_clusters=3, sep_degree=s, n_noisy=noise, min_cluster_size=30, 
                                 max_cluster_size=100, outliers = 0)
        X.append(XX)
        y.append(yy)

## Interactive testing

In [13]:
@interact(DATA = (1,6,1), NEIGHBORS = (3, 15, 2), OUTLIERS = (0.1, 0.4))
def g(DATA = 1, NEIGHBORS=3, OUTLIERS=0.1):
    lof = LocalOutlierFactor(n_neighbors=NEIGHBORS, contamination=OUTLIERS)    
    labels = lof.fit_predict(X[DATA-1])
    count_correct = 0
    count_outliers = 0.
    count_incorrect = 0
    for i in range(len(y[DATA-1])):
        if np.isnan(y[DATA-1][i]):
            count_outliers += 1
            if labels[i] == -1:
                count_correct += 1
        elif labels[i] == -1:
            count_incorrect += 1
    total = count_correct + count_incorrect
    print("Correct outliers: ", count_correct/count_outliers, "(",count_correct,"/",count_outliers,")")
    print("Incorrect outliers: ", count_incorrect/total, "(",count_incorrect,"/",total,")")
    fig = plt.figure(figsize=(16,6))
    plt.subplot(1,2,1)
    plt.scatter(X[DATA-1][:, 2], X[DATA-1][:, 1], c=labels,s=100)
    plt.title('Outliers/Inliers')
    plt.subplot(1,2,2)
    plt.scatter(X[DATA-1][:, 2], X[DATA-1][:, 1], c=lof.negative_outlier_factor_,s=100)
    plt.title('LOF')

interactive(children=(IntSlider(value=1, description='DATA', max=6, min=1), IntSlider(value=3, description='NE…

In [14]:
@interact(DATA = (1, 12, 1))
def g(DATA=1):
    km = KMeans(n_clusters=3)
    km.fit(X[DATA-1])
    labels = km.predict(X[DATA-1])
    print(adjusted_mutual_info_score(y[DATA-1], labels))
    plt.figure(figsize=(10,10))
    plt.scatter(X[DATA-1][:, 2], X[DATA-1][:, 1], c=labels, s=100)
    plt.scatter(km.cluster_centers_[:,2], km.cluster_centers_[:,1], c='r', s=200);

interactive(children=(IntSlider(value=1, description='DATA', max=12, min=1), Output()), _dom_classes=('widget-…

In [15]:
@interact(DATA = (1, 12, 1), eps = (1, 11, 2), ms = (5, 50, 5))
def g(DATA = 1, eps=1, ms=5):
    dbs = DBSCAN(eps=eps, min_samples=ms)
    labels = dbs.fit_predict(X[DATA-1])
    unq = len(np.unique(labels))
    print("NClusters", unq-1)
    sc = adjusted_mutual_info_score(y[DATA-1], labels)
    print(sc)
    ecolors = np.array(labels)
    ecolors[ecolors == -1] += unq+25
    plt.figure(figsize=(10,10))
    plt.scatter(X[DATA-1][:, 0], X[DATA-1][:, 1], c=ecolors/unq+25, s=50);
    return sc

interactive(children=(IntSlider(value=1, description='DATA', max=12, min=1), IntSlider(value=1, description='e…

In [16]:
@interact(DATA = (1, 12, 1))
def g(DATA = 1):
    cons = SimpleConsensusClustering(n_clusters=3, n_clusters_base=10, n_components=30, ncb_rand=False)
    cons.fit(X[DATA-1])
    labels = cons.labels_
    print('SCC AMI =', adjusted_mutual_info_score(labels,y[DATA-1]))
    fig = plt.figure(figsize=(20,7))
    ax = fig.add_subplot(121)
    plt.scatter(X[DATA-1][:,0],X[DATA-1][:,1],c=y[DATA-1])
    ax = fig.add_subplot(122)
    plt.scatter(X[DATA-1][:,0],X[DATA-1][:,1],c=labels);

interactive(children=(IntSlider(value=1, description='DATA', max=12, min=1), Output()), _dom_classes=('widget-…